In [ ]:
!pip install pymp-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pymp-pypi: filename=pymp_pypi-0.5.0-py3-none-any.whl size=10339 sha256=3f61d89ce85a4b700ad26ab8e44f1eaf121aa3e5ec1e00f6c6ddd9bcc59dabce
  Stored in directory: /root/.cache/pip/wheels/22/2a/4e/d49c406bb5eb2c04b424940de41d40b3b1481c31b9f93a13c1
Successfully built pymp-pypi


In [ ]:
from __future__ import print_function

import pymp
import random
import numpy as np
import time
from sympy import isprime


Linear Search

In [ ]:
#%%timeit
def linearsearch(arr,search_var):
  count=0
  for i in range(len(arr)):
    if arr[i]==search_var:
      count+=1
  return count

number_to_search=5

input_array=[random.randint(1,10) for i in range(100)]
array_len=len(input_array)


count = linearsearch(input_array,number_to_search)

print(count)

11


In [ ]:


number_to_search=5
array_len=100

ex_array = pymp.shared.array((array_len,), dtype='uint8')
for index in range(0, array_len):
  ex_array[index] = random.randint(1,10)

count = 0

with pymp.Parallel(4) as p:
  for index in p.range(0,array_len):
    if ex_array[index]==number_to_search:
      #print('Number ',number_to_search,' found at ',index,' position')
      count+=1

print(count)

2


Check Prime Number

In [ ]:
%%time
arr_len=1000

data=pymp.shared.array((arr_len,),dtype='uint8')
res=pymp.shared.array((1,),dtype='uint8')

def isPrime(n) :
	if (n <= 1) :
		return False
	if (n <= 3) :
		return True
	if (n % 2 == 0 or n % 3 == 0) :
		return False
	i = 5
	while(i * i <= n) :
		if (n % i == 0 or n % (i + 2) == 0) :
			return False
		i = i + 6

	return True

def check_prime(arr):
  res_s=[]
  for i in arr:
    if isPrime(i):
      res_s.append(True)
    else:
      res_s.append(False)
  return res_s




res_seq=check_prime(data)

CPU times: user 1.14 ms, sys: 1.03 ms, total: 2.17 ms
Wall time: 2.13 ms


In [ ]:


for i in range(arr_len):
  data[i]=random.randint(0,9)

with pymp.Parallel(4) as p:
  for index in p.range(arr_len):
    if isPrime(data[index])==True:
      res=np.append(res,np.array([True]))
    else:
      res=np.append(res,np.array([False]))

Matrix Multiplication

In [ ]:

size = 99
mat1 = np.random.randint(1,100000,(size,size))
mat2 = np.random.randint(1,100000,(size,size))


res = [[0 for x in range(mat1.shape[0])] for y in range(mat2.shape[0])]

for i in range(len(mat1)):
    for j in range(len(mat2[0])):
        for k in range(len(mat2)):

            res[i][j] += mat1[i][k] * mat2[k][j]


In [ ]:

ex_array = pymp.shared.array((mat1.shape[0],mat1.shape[0]), dtype='uint8')

start=time.time()
#only for outer loop
with pymp.Parallel(4) as p:
  for i in p.range(len(mat1)):
    for j in range(len(mat2[0])):
      for k in range(len(mat2)):
        ex_array[i][j]+= mat1[i][k] * mat2[k][j]
print("For Outer Loop: ",time.time()-start)


start=time.time()
#only for inner loop
with pymp.Parallel(4) as p:
  for i in range(len(mat1)):
    for j in p.range(len(mat2[0])):
      for k in p.range(len(mat2)):
        ex_array[i][j]+= mat1[i][k] * mat2[k][j]
print("For Inner Loop: ",time.time()-start)


#For all loops
start=time.time()
with pymp.Parallel(4) as p:
  for i in p.range(len(mat1)):
    for j in p.range(len(mat2[0])):
      for k in p.range(len(mat2)):
        ex_array[i][j]+= mat1[i][k] * mat2[k][j]
print("For All Loops: ",time.time()-start)

print(ex_array.shape)


For Outer Loop:  2.437717914581299
For Inner Loop:  0.8155877590179443
For All Loops:  0.20045828819274902
(99, 99)


Check Prime Number in PowerSet


In [ ]:
def powerset(s):
  x = len(s)
  p_list=[]
  for i in range(1 << x):
    p_list.append([s[j] for j in range(x) if (i & (1 << j))])
  return p_list
powerset_list=powerset([i for i in range(1,25)])


In [ ]:
def checkprime(arr):
  arrsum=0
  res=[]
  for i in arr:
      res.append(isprime(sum(i)))
  return res


In [ ]:
start=time.time()
single_process_res=checkprime(powerset_list)
print('Single process takes ',time.time()-start)


Single process takes  12.445126056671143


In [ ]:
len_list=len(powerset_list)
ex_array = pymp.shared.array((1,), dtype='uint8')
start=time.time()

#complete the pymp parallel code here
with pymp.Parallel(4) as p:
  def powerset1(s):
    x = len(s)
    p_list=[]
    for i in p.range(1 << x):
      p_list.append([s[j] for j in p.range(x) if (i & (1 << j))])
    return p_list
  powerset_list1=powerset1([i for i in p.range(1,25)])
  def checkprime1(arr):
    arrsum=0
    res=[]
    for i in arr:
      res.append(isprime(sum(i)))
    return res
  t=checkprime1(powerset_list1)
print('Parallel Process takes ',time.time()-start)
print(t)


Parallel Process takes  0.14502286911010742
[False, False, True, True, False, False, True, True, False, False, True, True, False, False, True, True]
